In [1]:
import unittest
import pandas as pd
import numpy as np
import tempfile
from bayesian_testing import BayesianMultivariateTest

class TestBayesianMultivariateTest(unittest.TestCase):
    def setUp(self):
        """Create a temporary CSV file with test data."""
        self.temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        self.data = pd.DataFrame({
            "Combination": ["A", "B","C", "D"],
            "Conversions": [10, 20, 25, 19],
            "Visitors": [200, 200, 200, 200]
        })
        self.data.to_csv(self.temp_file.name, index=False)

        # Initialize BayesianMultivariateTest instance
        self.test_instance = BayesianMultivariateTest(self.temp_file.name)

    def test_validate_data(self):
        """Test if the validate_data method correctly identifies missing columns."""
        # Create incorrect data
        incomplete_data = pd.DataFrame({
            "Combination": ["A", "B"],
            "Conversions": [10, 20]
        })
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".csv")
        incomplete_data.to_csv(temp_file.name, index=False)

        with self.assertRaises(ValueError) as context:
            BayesianMultivariateTest(temp_file.name)
        self.assertIn("Missing required column", str(context.exception))

    def test_calculate_posteriors(self):
        """Test if posteriors are correctly computed."""
        posteriors = self.test_instance.calculate_posteriors()
        self.assertIsInstance(posteriors, dict)
        self.assertEqual(len(posteriors), 4)  # Four combinations

    def test_update_prior_with_posterior(self):
        """Test if priors update correctly when data changes."""
        self.test_instance.calculate_posteriors()

        old_alpha = self.test_instance.prior_alpha
        old_beta = self.test_instance.prior_beta

        # Modify data
        new_data = pd.DataFrame({
            "Combination": ["A", "B",],
            "Conversions": [15, 25],  
            "Visitors": [110, 210]  
        })
        new_data.to_csv(self.temp_file.name, index=False)
        self.test_instance.data = pd.read_csv(self.temp_file.name)
        self.test_instance.calculate_posteriors()
        self.test_instance.update_prior_with_posterior()

        self.assertNotEqual(self.test_instance.prior_alpha, old_alpha)
        self.assertNotEqual(self.test_instance.prior_beta, old_beta)

    def test_probability_of_being_best(self):
        """Test if probability_of_being_best returns correct probabilities."""
        self.test_instance.calculate_posteriors()
        probabilities = self.test_instance.probability_of_being_best()

        self.assertIsInstance(probabilities, pd.Series)
        self.assertEqual(len(probabilities), 4)  # Four combinations

    def test_summary(self):
        """Test if the summary method returns the correct dataframe."""
        summary = self.test_instance.summary()

        self.assertIsInstance(summary, pd.DataFrame)
        self.assertIn("Probability of Being Best", summary.columns)
        self.assertEqual(len(summary), 4)  # Four combinations

    def test_get_posterior_summary(self):
        """Test if posterior summary returns correct statistics."""
        self.test_instance.calculate_posteriors()
        posterior_summary = self.test_instance.get_posterior_summary()

        self.assertIsInstance(posterior_summary, dict)
        for comb in posterior_summary:
            self.assertIn("mean", posterior_summary[comb])
            self.assertIn("95% CI lower", posterior_summary[comb])
            self.assertIn("95% CI upper", posterior_summary[comb])

    def test_estimate_days_to_run_exp(self):
        """Test if estimate_days_to_run_exp returns a reasonable number of days."""
        days = self.test_instance.estimate_days_to_run_exp(daily_visitors=50)
        self.assertIsInstance(days, int)
        self.assertGreaterEqual(days, 1)
        self.assertLessEqual(days, 14)

if __name__ == '__main__':
    unittest.main(argv= [''], exit = False)


.

Combination: A, Posterior: Beta(11.00, 191.00)
Combination: B, Posterior: Beta(21.00, 181.00)
Combination: C, Posterior: Beta(26.00, 176.00)
Combination: D, Posterior: Beta(20.00, 182.00)


......
----------------------------------------------------------------------
Ran 7 tests in 1.131s

OK


Combination: A, Posterior: Beta(11.00, 191.00)
Combination: B, Posterior: Beta(21.00, 181.00)
Combination: C, Posterior: Beta(26.00, 176.00)
Combination: D, Posterior: Beta(20.00, 182.00)
Combination: A, Posterior: Beta(11.00, 191.00)
Combination: B, Posterior: Beta(21.00, 181.00)
Combination: C, Posterior: Beta(26.00, 176.00)
Combination: D, Posterior: Beta(20.00, 182.00)
Combination: A, Posterior: Beta(11.00, 191.00)
Combination: B, Posterior: Beta(21.00, 181.00)
Combination: C, Posterior: Beta(26.00, 176.00)
Combination: D, Posterior: Beta(20.00, 182.00)
Combination: A, Posterior: Beta(11.00, 191.00)
Combination: B, Posterior: Beta(21.00, 181.00)
Combination: C, Posterior: Beta(26.00, 176.00)
Combination: D, Posterior: Beta(20.00, 182.00)
Combination: A, Posterior: Beta(16.00, 96.00)
Combination: B, Posterior: Beta(26.00, 186.00)
Priors updated based on new data.
